In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Regressors
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#Metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
%matplotlib inline
#Clustering
from sklearn.cluster import KMeans 
from sklearn.cluster import DBSCAN
#Otros
from sklearn.multioutput import MultiOutputRegressor

### CSV Loading

In [103]:
dataset = pd.read_csv('./data/t_data/anime_cleaned_treated_v2_4_b.csv')

In [104]:
dataset.head()

,Genres,Source,Type,Episodes,Decimal,Romance,Shoujo,Ecchi,Shounen,Fantasy,...,Mistery,Thriller,Supernatural,Gore,SciFi,Action,Otro,Members,Score,Favorites
0,"Comedy, Romance, Shounen, Supernatural",6,5,12,147,1,0,0,1,0,...,0,0,1,0,0,0,0,283882,7.63,2809
1,"Comedy, Romance, Shounen",6,5,26,19,1,0,0,1,0,...,0,0,0,0,0,0,0,204003,7.89,2579
2,"Comedy, Romance, Fantasy, Shoujo",6,5,51,519,1,1,0,0,1,...,0,0,0,0,0,0,0,70127,7.55,802
3,"Comedy, Romance, Fantasy",9,5,38,7,1,0,0,0,1,...,0,0,0,0,0,0,0,93312,8.21,3344
4,"Comedy, Romance, Shounen",6,5,25,19,1,0,0,1,0,...,0,0,0,0,0,0,0,182765,8.67,2082


### Training

In [105]:
data = dataset.values
X = data[:int(len(data)*0.7),1:22]
y = data[:int(len(data)*0.7),22:24]
X_t = data[int(len(data)*0.7):,1:22]
y_t = data[int(len(data)*0.7):,22:24]
print y[0]

[283882 7.63]


In [106]:
DT_R = DecisionTreeRegressor(max_depth=5)
RF_R = RandomForestRegressor(n_estimators=40)
GB_R = MultiOutputRegressor(GradientBoostingRegressor())
#np.array([6,5,12,147]).reshape(1,-1)

In [107]:
DT_R.fit(X,y)
RF_R.fit(X,y)
GB_R.fit(X,y)

MultiOutputRegressor(estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
           n_jobs=1)

In [108]:
print np.min(dataset['Members'])
print np.max(dataset['Members'])
m = dataset['Members'].sort_values()
print np.mean([abs(m[i]-m[i-1]) for i in range(1,len(m))])

18
1456378
70579.2284386


In [109]:
for i in range(1):
    print "============="
    print RF_R.predict(X_t[i].reshape(1,-1))
    print DT_R.predict(X_t[i].reshape(1,-1))
    print GB_R.predict(X_t[i].reshape(1,-1))
    print y_t[i]

[[  7.85234250e+04   7.27295833e+00]]
[[  9.10855379e+04   7.19682759e+00]]
[[  1.27706730e+05   7.44853863e+00]]
[699351 8.56]


In [110]:
s = cross_val_score(DT_R, X_t, y_t, scoring=make_scorer(mean_squared_error))
print "MSE %.3f (+/- %.5f)"%(np.sqrt(np.mean(s)), np.sqrt(np.std(s)))

MSE 63670.439 (+/- 26003.35061)


In [111]:
s = cross_val_score(RF_R, X_t, y_t, scoring=make_scorer(mean_squared_error))
print "MSE %.3f (+/- %.5f)"%(np.sqrt(np.mean(s)), np.sqrt(np.std(s)))

MSE 62999.349 (+/- 26997.78543)


In [112]:
s = cross_val_score(GB_R, X_t, y_t, scoring=make_scorer(mean_squared_error))
print "MSE %.3f (+/- %.5f)"%(np.sqrt(np.mean(s)), np.sqrt(np.std(s)))

MSE 62499.640 (+/- 29783.58133)


### Arbitrary new anime prediction test

In [114]:
new_anime = np.array([6,5,24,50822,1,1,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0]).reshape(1,-1)

In [115]:
GB_R.predict(new_anime)

array([[  1.57893100e+05,   6.90995925e+00]])

### Object persistence

In [117]:
import pickle
with open('regressors.pkl', 'wb') as output:
    pickle.dump(DT_R, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(RF_R, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(GB_R, output, pickle.HIGHEST_PROTOCOL)
